# Notebook to create a SDL configuration from a list of tables and primary keys


In the first cell the parameters are given. In the second cell the configuration template is defined. In the third cell the templates are merged, printed and written to file.


Note: in some templates are variables used. These strings start with `f'''`. In these, we need to use double curly brackets: `{{` instead of `{` and `}}` instead of `}`. Single brackets are used for variables.

<br>

TODO: 

- (ext) Gotama -> (stg) as parquet files -> (int) DeltaLakeTableDataObjects

-> add dataObjects and Actions

seperate global settings to separate *.conf and remove from this script




In [1]:

eol='\n'
tables= {
        "LeiPKopf": "",
        "LeiPPos": "lpp_id", 
        "Deckung": "",
        "VersDeckung": "", 
        "Tariffaktor": "",
        "Familie": "",
        "Gemeinde": "",
        "Aufenthaltsart": "",
        "Versicherter": "ver_nr",
        "ECKopf": "",
        "ECDetail": "",
        "ECProdukt": "",
        "VersDeckung": "", 
        "Aufenthaltsart": "",
        "Schadenart": "",
    }
filename="/mnt/config/GotamaAutoCreated.conf"
srcType="JdbcTableDataObject"
stgType="ParquetFileDataObject"
intType="DeltaLakeTableDataObject"
connectionID="GotamaSql"



In [2]:
def pre():
    return '''
global {
  spark-options {
    "spark.hadoop.javax.jdo.option.ConnectionURL" = "jdbc:derby://metastore:1527/db;create=true"
    "spark.hadoop.javax.jdo.option.ConnectionDriverName" = "org.apache.derby.jdbc.ClientDriver"
    "spark.hadoop.javax.jdo.option.ConnectionUserName" = "sa"
    "spark.hadoop.javax.jdo.option.ConnectionPassword" = "1234"
    "spark.sql.shuffle.partitions" = 2
    "spark.databricks.delta.snapshotPartitions" = 2
  }
}

connections {
  GotamaSql {
    type = JdbcTableConnection
    url = "jdbc:sqlserver://mssqlserver:1433;encrypt=true;trustServerCertificate=true;database=GotamaMaster"
    driver = com.microsoft.sqlserver.jdbc.SQLServerDriver
    db = ""
    authMode {
      type = BasicAuthMode
      userVariable = "CLEAR#sa"
      passwordVariable = "CLEAR#%abcd1234%"
    }
  }
}
'''

def DAprolog ():
    return '''
dataObjects {
'''

def extDataObject (name):
    return f'''
  ext-{name} {{
    type = {srcType}
    connectionId = {connectionID}
    table = {{
      name = "{name}"
    }}
  }}
'''

def stgDataObject (name, primKey):
    primK = f"{eol}      primaryKey = [{primKey}]" if primKey != "" else ""
    return f'''
  stg-{name} {{
    type = {stgType}
    path = "~{{id}}"
    table = {{
      db = "default"
      name = "stg_{name.replace("-","_")}"{primK}
    }}
  }}
'''

def DAepilog ():
    return '''
}
'''

def ActionProlog ():
    return '''
actions {
'''

def Action (name):
    return f'''
  GOTAMA-{name} {{
    type = CopyAction
    inputId = ext-{name}
    outputId = stg-{name}
    metadata {{
      feed = Gotama-download
    }}
  }}
'''

def ActionEpilog ():
    return '''
}
'''

In [3]:
content = ( 
    pre() +
    DAprolog() 
)
for tab,primKey in tables.items():
  content += ( 
    extDataObject(tab) + 
    stgDataObject(tab, primKey)
  )

content += ( DAepilog() + ActionProlog() )
for tab,primKey in tables.items():
  content += Action(tab)
content += ActionEpilog()

print(content)
f = open(filename, "w")
f.write(content)
f.close()


global {
  spark-options {
    "spark.hadoop.javax.jdo.option.ConnectionURL" = "jdbc:derby://metastore:1527/db;create=true"
    "spark.hadoop.javax.jdo.option.ConnectionDriverName" = "org.apache.derby.jdbc.ClientDriver"
    "spark.hadoop.javax.jdo.option.ConnectionUserName" = "sa"
    "spark.hadoop.javax.jdo.option.ConnectionPassword" = "1234"
    "spark.sql.shuffle.partitions" = 2
    "spark.databricks.delta.snapshotPartitions" = 2
  }
}

connections {
  GotamaSql {
    type = JdbcTableConnection
    url = "jdbc:sqlserver://mssqlserver:1433;encrypt=true;trustServerCertificate=true;database=GotamaMaster"
    driver = com.microsoft.sqlserver.jdbc.SQLServerDriver
    db = ""
    authMode {
      type = BasicAuthMode
      userVariable = "CLEAR#sa"
      passwordVariable = "CLEAR#%abcd1234%"
    }
  }
}

dataObjects {

  ext-LeiPKopf {
    type = JdbcTableDataObject
    connectionId = GotamaSql
    table = {
      name = "LeiPKopf"
    }
  }

  stg-LeiPKopf {
    type = DeltaLakeTableDat